<a href="https://colab.research.google.com/github/abhilashpandurangan/RASRCustomerSupport/blob/master/basic_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Seq2Seq
seq2seq model implementation using keras, Tensorflow to predict company responses to consumers. 

Source:
https://github.com/google/seq2seq

![seq2seq model architecture](https://i.imgur.com/JmuryKu.png)

In [0]:
import re
import random
import time

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')
import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize

from tqdm import tqdm_notebook as tqdm

## Model Parameters

In [0]:
MAX_VOCAB_SIZE = 2**13  # 8192
MAX_MESSAGE_LEN = 30  
# Embedding size for words 
EMBEDDING_SIZE = 100
CONTEXT_SIZE = 100
BATCH_SIZE = 4
DROPOUT = 0.2

LEARNING_RATE=0.005
# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted
SUB_BATCH_SIZE = 1000


## Data Prep
Here, we'll prepare the data for training our seq2seq model, including:

- Replace screen names with `@__sn__` token to show model the commonality between them
- Build a vocab to turn tokens into integers suitable for our seq2seq model
- Tokenize input and target text into fixed size vectors
- Partition our dataset into train and test sets

### Data Loading and Reshaping


In [0]:
%%time
tweets = pd.read_csv('/content/drive/My Drive/NLP CSCI 544 Project/code/twcs.csv')

first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]

inbounds_and_outbounds = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id').sample(frac=1)

# Filter to only outbound replies (from companies)
inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]

tqdm().pandas()  # Enable tracking of progress in dataframe `apply` calls

In [10]:
print(f'Data shape: {inbounds_and_outbounds.shape}')

Data shape: (794299, 14)


### Tokenizing and Vocab Build

 NLTK's `casual_tokenize`, which handles a lot of corner cases found in social media data ("casual" text data) along with scitkit learn's `CountVectorizer`.  

In [11]:
inbounds_and_outbounds.head()

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
4698,17153,119779,True,Wed Nov 01 12:21:26 +0000 2017,@idea_cares still no help about this. https://...,17152,NaN,17152,idea_cares,False,Wed Nov 01 12:37:28 +0000 2017,@119779 We understand the wait has been long f...,NaN,17153.0
446262,1613091,494932,True,Sat Nov 25 18:13:34 +0000 2017,@116062 why can’t we get answers on transactio...,1613090,NaN,1613090,AskTarget,False,Sun Nov 26 04:07:12 +0000 2017,@494932 Thank you for reaching out to us. We'd...,NaN,1613091.0
241783,893045,332047,True,Fri Oct 13 17:49:48 +0000 2017,@Delta 2nd time in 3 weeks you delayed/cancele...,893044,NaN,893044,Delta,False,Fri Oct 13 18:06:19 +0000 2017,"@332047 Hello, Paul! I am sorry that we have d...",NaN,893045.0
311567,1138117,388074,True,Sat Oct 14 18:52:17 +0000 2017,@388075 Eden Mill Blueberry &amp; Vanilla Gin...,"1138116,1138118",NaN,1138116,AldiUK,False,Sat Oct 14 19:03:34 +0000 2017,@388074 @388075 All the ingredients for a perf...,NaN,1138117.0
833432,2851326,792799,True,Mon Nov 27 18:49:54 +0000 2017,@JetBlue for the last 3 days I get ERR when tr...,2851324,NaN,2851324,JetBlue,False,Mon Nov 27 18:51:02 +0000 2017,@792799 Oh no! Are you trying on your desktop ...,2851325,2851326.0


In [12]:
# Replace anonymized screen names with common token @__sn__
def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return '@__sn__'

sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')
print("Replacing anonymized screen names in X...")
x_text = inbounds_and_outbounds.text_x.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))
print("Replacing anonymized screen names in Y...")
y_text = inbounds_and_outbounds.text_y.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))

Replacing anonymized screen names in X...




Replacing anonymized screen names in Y...


In [13]:
count_vec = CountVectorizer(tokenizer=casual_tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit(tqdm(x_text + y_text))
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START
# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

Fitting CountVectorizer on X and Y text data...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"



Learned vocab of 8192 items.


### Vocab Helper Functions
These helper functions take strings and turn them into word indexes used by the actual seq2seq models.  apply the `to_word_idx` function to our text data to get our `N x MESSAGE_LEN` training/test data.

In [0]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()


In [15]:
# Make sure our helpers work as expected...
x_text.head().apply(to_word_idx).apply(from_word_idx)

4698         @idea_cares still no help about this . __unk__
446262    @__sn__ why can ’ t we get answers on transact...
241783    @delta 2nd time in 3 weeks you delayed / cance...
311567    @__sn__ __unk__ __unk__ __unk__ & __unk__ gin ...
833432    @jetblue for the last 3 days i get __unk__ whe...
Name: text_x, dtype: object

In [16]:
print("Calculating word indexes for X...")
x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

Calculating word indexes for X...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  



Calculating word indexes for Y...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.


### Train / Test Split
Here, we split our data into training and test sets.  80-20 split for train-test

In [17]:
all_idx = list(range(len(x)))
train_idx = set(random.sample(all_idx, int(0.8 * len(all_idx))))
test_idx = {idx for idx in all_idx if idx not in train_idx}

train_x = x[list(train_idx)]
test_x = x[list(test_idx)]
train_y = y[list(train_idx)]
test_y = y[list(test_idx)]

assert train_x.shape == train_y.shape
assert test_x.shape == test_y.shape

print(f'Training data of shape {train_x.shape} and test data of shape {test_x.shape}.')

Training data of shape (635439, 30) and test data of shape (158860, 30).


## Model Creation


- Shared word embeddings

- Encoder RNN
  
- Decoder RNN
  
- Next Word Dense+Softmax
  


In [0]:

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
    TimeDistributed
from keras.utils import np_utils

In [0]:
#!pip install tensorflow==1.4.0
#!pip uninstall keras
#!pip install keras==2.1.2

In [0]:
def create_model():
    shared_embedding = Embedding(
        output_dim=EMBEDDING_SIZE,
        input_dim=MAX_VOCAB_SIZE,
        input_length=MAX_MESSAGE_LEN,
        name='embedding',
    )
    
    # ENCODER
    
    encoder_input = Input(
        shape=(MAX_MESSAGE_LEN,),
        dtype='int32',
        name='encoder_input',
    )
    
    embedded_input = shared_embedding(encoder_input)
    
    # No return_sequences - since the encoder here only produces a single value for the
    # input sequence provided.
    encoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='encoder',
        dropout=DROPOUT
    )
    
    context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
    
    # DECODER
    
    last_word_input = Input(
        shape=(MAX_MESSAGE_LEN, ),
        dtype='int32',
        name='last_word_input',
    )
    
    embedded_last_word = shared_embedding(last_word_input)
    # Combines the context produced by the encoder and the last word uttered as inputs
    # to the decoder.
    decoder_input = concatenate([embedded_last_word, context], axis=2)
    
    # return_sequences causes LSTM to produce one output per timestep instead of one at the
    # end of the intput, which is important for sequence producing models.
    decoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='decoder',
        return_sequences=True,
        dropout=DROPOUT
    )
    
    decoder_output = decoder_rnn(decoder_input)
    
    # TimeDistributed allows the dense layer to be applied to each decoder output per timestep
    next_word_dense = TimeDistributed(
        Dense(int(MAX_VOCAB_SIZE / 2), activation='relu'),
        name='next_word_dense',
    )(decoder_output)
    
    next_word = TimeDistributed(
        Dense(MAX_VOCAB_SIZE, activation='softmax'),
        name='next_word_softmax'
    )(next_word_dense)
    
    return Model(inputs=[encoder_input, last_word_input], outputs=[next_word])

s2s_model = create_model()
optimizer = Adam(lr=LEARNING_RATE, clipvalue=5.0)
s2s_model.compile(optimizer='adam', loss='categorical_crossentropy')

## Model Training


In [0]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])

In [0]:
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])

In [0]:
def train_mini_epoch(model, start_idx, end_idx):

    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    print('Test results:', model.evaluate(
        [test_x[rand_idx], add_start_token(test_y[rand_idx])],
        binarize_labels(test_y[rand_idx])
    ))
    
    input_strings = [
        "@AppleSupport I fix I this I stupid I problem I",
        "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service.",
    ]
    
    for input_string in input_strings:
        output_string = respond_to(model, input_string)
        print(f'> "{input_string}"\n< "{output_string}"')


### Train the model!


In [24]:
training_time_limit = 360 * 60  # seconds (notebooks terminate after 1 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(100):
        print(f'Training in epoch {epoch}...')
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

Training in epoch 0...
Epoch 1/1
1000/1000 [==============================] - 100s 100ms/step
Test results: 4.230677749633789
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi , , , , , , , , , , . ."
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi , , , , , , , , , , . ."
Epoch 1/1
1000/1000 [==============================] - 100s 100ms/step
Test results: 3.844701509475708
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi , sorry to hear to hear to the __unk__ . __unk__ . __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi , sorry to hear to hear to hear to the __unk__ . __unk__ . __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __unk__ __

In [25]:
respond_to(s2s_model, '''@AppleSupport iPhone 8 touchID doesnt unlock while charging on 
    110v w/ 61w laptop charger to usbc lightning cable just uh.. so you guys know''')

"@__sn__ hi , __unk__ . we are sorry to hear this . please dm us your idea number and we'll be happy to help you . ^ __unk__"

In [28]:
# serialize model to JSON
model_json = s2s_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
s2s_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [30]:
 #load json and create model
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")


Loaded model from disk
